In [1]:
import sys
sys.path.append('/home/prometeo/Programacion/nanorag')
from base import *
from store import *
from utils import *
import PyPDF2

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", device_map = "cuda")
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB (GPU 0; 23.66 GiB total capacity; 8.37 GiB already allocated; 94.25 MiB free; 8.45 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
class LLM:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def __call__(self, prompt, max_length=100):
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids.to(self.device)
        output_ids = self.model.generate(input_ids=input_ids, max_length=max_length)
        return self.tokenizer.decode(output_ids[0], skip_special_tokens=True).strip(prompt)

In [ ]:
llm = LLM(model = model, tokenizer = tokenizer)

In [ ]:
class PromptTemplate:
    def __init__(self, template = 'A user provided this instructions'):
        self.template = template

    def __call__(self, input_text):
        self.prompt = f"{self.template}: {input_text} Output:"
        return self.prompt

prompt_template = PromptTemplate(template = 'Eres un experto en el espanol y siempre responderas en espanol a cualquier consulta. A continuacion un usuario te hara una consulta en espanol, que responderas con elocuencia y certeza')
input_text = 'Quien es la mujer mas perra del mundo, me refiero a una mujer, que sea considerada infiel o maleducada'
prompt = prompt_template(input_text)


In [ ]:
response = llm(prompt = prompt, max_length = 1500)
print(response)

In [ ]:
context = ModelContext(llm = llm, embedding = embedding_model, tokenizer = tokenizer)

### Loaders

#### PDF Downloader

In [ ]:
import requests
from bs4 import BeautifulSoup
import httpx
async def scrape_links(url):
    async with httpx.AsyncClient(timeout = 100) as client:
        try:
            response = await client.get(url)
            response.raise_for_status()  # Ensure we got a successful response
        except httpx.HTTPStatusError as e:
            if e.response.status_code == 429:
                print(f"Rate limit exceeded when fetching {url}")
                retry_after = e.response.headers.get('Retry-After')
                if retry_after:
                    print(f"Server suggests retrying after {retry_after} seconds")
            else:
                print(f"Unable to fetch {url}, status code: {e.response.status_code}")
            return set()
        except httpx.ConnectError:
            print(f"Unable to connect to {url}")
            return set()
        except:
            print('Unknown exception ocurred')
            return set()

    soup = BeautifulSoup(response.text, 'html.parser')

    links = set()
    for a in soup.find_all('a', href=True):
        links.add(a['href'])

    return links

In [ ]:
#I want to get a lot of files from this webpages to do RAG on them and analyze them at scale.
# I want to get a huge dataset of pdfs, like 500/1000 reelevant papers or so
urls = ["https://paperswithcode.com/task/language-modelling", "https://paperswithcode.com/task/question-answering", "https://paperswithcode.com/task/contrastive-learning","https://paperswithcode.com/task/object-detection","https://paperswithcode.com/task/medical-image-segmentation","https://paperswithcode.com/task/transfer-learning","https://paperswithcode.com/task/recommendation-systems","https://paperswithcode.com/task/representation-learning","https://paperswithcode.com/task/time-series","https://paperswithcode.com/task/node-classification","https://paperswithcode.com/task/graph-embedding","https://paperswithcode.com/task/speech-recognition"]

In [ ]:
import asyncio,re

In [ ]:
split_at_single_slash = lambda s: re.split(r'(?<!/)/(?!/)', s)

In [ ]:
url_dict = {url: links for url, links in zip(urls, await asyncio.gather(*(scrape_links(url) for url in urls)))}

url_clean_set = set()
keywords = ['/paper/']
for url, links in url_dict.items():  
    url_to_add = split_at_single_slash(url)[0]
    filtered_urls = set(url_to_add + paper for paper in links if any(keyword in paper for keyword in keywords))
    url_clean_set.update(filtered_urls)
len(url_clean_set)

In [ ]:
tasks = [scrape_links(link) for link in url_clean_set]
results = await asyncio.gather(*tasks)

In [ ]:
result_set = {element for subset in results for element in subset}

In [ ]:
# Scrape the links from the URLs in parallel
#A dict of set can also save a lot of time
keywords = ['pdf','.pdf']
filtered_pdfs = set()
for link in result_set:
    if any(keyword in link for keyword in keywords):
        filtered_pdfs.add(link)
len(filtered_pdfs)

In [ ]:
split_at_single_slash(list(filtered_pdfs)[0])[-1] # for each one download 

In [ ]:
url_2 = 'https://arxiv.org/list/cs.AI/pastweek?show=600'

In [ ]:
url_to_add = split_at_single_slash(url_2)[0]

In [ ]:
links = await scrape_links(url_2)

In [ ]:
keywords = ['pdf','.pdf']
for link in links:
    if any(keyword in link for keyword in keywords):
        filtered_pdfs.add(url_to_add + link)

In [ ]:
pdf_list = list(filtered_pdfs) #epic shit

In [ ]:
import pickle
with open('pdf_urls.pkl', 'wb') as f:
    pickle.dump(pdf_list, f)

#### Now downloading all the pdfs of this links to separate folders in an async manner. 

In [ ]:
from pathlib import Path

In [ ]:
path_dir = Path('sample_docs')

In [ ]:
tasks = []
for link in pdf_list:
    file_path = path_dir / split_at_single_slash(link)[-1]
    task = download_pdf(link, file_path)
    tasks.append(task)
await asyncio.gather(*tasks)

In [ ]:
path_dir

In [ ]:
pdf_paths = [path for path in path_dir.iterdir() if '.pdf' in path.name]
pdf_paths_to_clean = [path for path in path_dir.iterdir() if '.pdf' not in path.name]

In [ ]:
len(pdf_paths)

Now I have 762 valid pdfs. 

In [ ]:
sample_file = pdf_paths_to_clean[0]

In [ ]:
links_2 = set()

In [ ]:
for path in pdf_paths_to_clean:
    try:
        with open(path, 'r') as f:
            contents = f.read()
    except:
        print('invalid file')
    try:
        soup = BeautifulSoup(contents, 'html.parser')
    except:
        print('Not html found')
    for a in soup.find_all('a', href=True):
        links_2.add(a['href'])

In [ ]:
soup = BeautifulSoup(contents, 'html.parser')
links_2 = set()
for a in soup.find_all('a', href=True):
    links_2.add(a['href'])


Cleaning and getting the ones that have been moved

In [ ]:
async def download_pdf(url, filename):
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url)
            with open(filename, 'wb') as f:
                f.write(response.content)
        except httpx.HTTPStatusError as e:
            if e.response.status_code == 429:
                print(f"Rate limit exceeded when fetching {url}")
                retry_after = e.response.headers.get('Retry-After')
                if retry_after:
                    print(f"Server suggests retrying after {retry_after} seconds")
            else:
                print(f"Unable to fetch {url}, status code: {e.response.status_code}")
            return set()
        except httpx.ConnectError:
            print(f"Unable to connect to {url}")
            return set()
        except:
            print('Unknown exception ocurred')
            return set()

#### PDF Loader

Making a downloader from arxiv links.

In [ ]:
from PyPDF2 import PdfReader
from pathlib import Path

In [ ]:
pdf_paths = [path for path in path_dir.iterdir() if '.pdf' in path.name]

In [ ]:
pdf_paths[0]

Want a way for example to easily download pdf files from a link. 

In [ ]:
file_path = pdf_paths[0]

In [ ]:
pdf_paths[0]

In [ ]:
output_dir = Path('extracted_images') / file_path.name.strip('.pdf')

In [ ]:
reader = PdfReader(file_path)

Get to know text templates in python

In [ ]:
#This is how we can extract and save images. 
count = 0
for page in reader.pages:
    for image_file_object in page.images:
        output_dir.mkdir(parents = True,exist_ok = True)
        with open(output_dir / f"{str(count)}{image_file_object.name}", "wb") as fp:
            fp.write(image_file_object.data, )
            count += 1

In [ ]:
text = ''
#Extracting text
for page in reader.pages:
    text = text + (page.extract_text())

In [ ]:
from base import Document, TextNode

In [ ]:
doc = Document(name = reader.metadata.title ,metadata = reader.metadata, text = text)

In [ ]:
nodes = doc.create_nodes_from_doc(model_context = context, category = 'PAPER')

In [ ]:
nodes[0]

In [ ]:
#Extracting metadata
meta = reader.metadata
print(len(reader.pages))

# All of the following could be None!
print(meta.author)
print(meta.creator)
print(meta.producer)
print(meta.subject)
print(meta.title)


**I will create a PDF Loader, markdown Loader, docx and txt loader**

#### 
